In [1]:
import re
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


In [2]:
aaaa = []
count = 1
browser = webdriver.Chrome('../chromedriver.exe')
url = 'https://www.cupoy.com/newsfeed/topstory'
browser.get(url)
time.sleep(10)
while True:
    
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load the page.
    time.sleep(1)

    # Calculate new scroll height and compare with last scroll height.
    browser.execute_script("return document.body.scrollHeight")
    
    # get infos
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    
    res = soup.find_all('div', class_='sc-eEieub sc-iuDHTM ibJqYc')
    # update infos (remove duplicate information!!)
    for i in res:
        aaaa.append(i)
        print('article_number is {}'.format(count))
        count +=1
    print(f"update... {len(aaaa)}")
    
    if len(aaaa) > 500:
        print("collected over")
        break
    time.sleep(5)   
browser.quit()
aaaa

article_number is 1
article_number is 2
article_number is 3
article_number is 4
article_number is 5
article_number is 6
article_number is 7
article_number is 8
article_number is 9
article_number is 10
article_number is 11
article_number is 12
article_number is 13
article_number is 14
article_number is 15
article_number is 16
article_number is 17
update... 17
article_number is 18
article_number is 19
article_number is 20
article_number is 21
article_number is 22
article_number is 23
article_number is 24
update... 24
article_number is 25
article_number is 26
article_number is 27
article_number is 28
article_number is 29
article_number is 30
article_number is 31
article_number is 32
article_number is 33
article_number is 34
article_number is 35
article_number is 36
article_number is 37
article_number is 38
article_number is 39
article_number is 40
article_number is 41
update... 41
article_number is 42
article_number is 43
article_number is 44
article_number is 45
article_number is 46
arti

article_number is 378
article_number is 379
article_number is 380
article_number is 381
article_number is 382
article_number is 383
article_number is 384
update... 384
article_number is 385
article_number is 386
article_number is 387
article_number is 388
article_number is 389
article_number is 390
article_number is 391
article_number is 392
article_number is 393
article_number is 394
article_number is 395
article_number is 396
article_number is 397
article_number is 398
article_number is 399
article_number is 400
article_number is 401
update... 401
article_number is 402
article_number is 403
article_number is 404
article_number is 405
article_number is 406
article_number is 407
article_number is 408
article_number is 409
article_number is 410
article_number is 411
article_number is 412
update... 412
article_number is 413
article_number is 414
article_number is 415
article_number is 416
article_number is 417
article_number is 418
article_number is 419
article_number is 420
article_numb

[<div class="sc-eEieub sc-iuDHTM ibJqYc" style="height: 204px; left: 0px; position: absolute; top: 2240px; width: 100%;"><a class="sc-RbTVP sc-kEmuub eSJXii" href="https://free.com.tw/savetweetvid/?utm_content=bufferae8f8&amp;utm_medium=social&amp;utm_source=facebook.com&amp;utm_campaign=buffer" target="_blank" title="SaveTweetVid 線上 Twitter 影片下載器，獲取 MP4 或轉 MP3、GIF"><div class="sc-hwcHae sc-fihHvN rQKGK" style='background-image: url("https://pubthumb.cupoy.com/httpsfree.com.twblogwpcontentuploads202003103417278m.png/small");'></div><div class="sc-lnmtFM sc-dznXNo gsZaZX"><div class="sc-hMrMfs cIcTrn"><div class="sc-drlKqa dBMTvt" style="width: 36px;"><div class="sc-bIqbHp eSyexw" style='left: 12px; border: 1px solid rgb(238, 238, 238); background-image: url("https://pubthumb.cupoy.com/httpsscontent.ftpe81.fna.fbcdn.netvt1.09543750101511343956839661260494886n.jpgohd8b13eb51eb6f5a209a76f324131fbf0oe5af0687f/icon");'></div><div class="sc-bIqbHp eSyexw" style='left: 0px; border: 1px solid 

In [53]:
browser = webdriver.Chrome('../chromedriver.exe')

#setting initial parameter 
url = 'https://www.cupoy.com/newsfeed/topstory'
#res_num = 0  #一個起始數字
switch = True
count = 1
data = {}
#執行讀取，等待回應
browser.get(url)
time.sleep(10)



#用來源網址做感測器勘誤

#設定文章數偵測器
while switch :
    soup = BeautifulSoup(browser.page_source,'html.parser')
    res = soup.find_all('div', class_='sc-eEieub sc-iuDHTM ibJqYc')
    
    #用迴圈處理資料
    for item in res:
        data.setdefault(count,{})
        det = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href')
        if det not in data:
            #分類或來源
            data[count]['category'] = item.find('div',{'class':'sc-gacfCG bPSpUf'}).string
            #文章來源網址
            data[count]['link'] = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href')
            #標題
            data[count]['title'] = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('title')
            #簡介
            data[count]['summary'] = item.find('p',{'class':'sc-FQuPU sc-ciodno bvnzOw'}).string

            #來源網址(若非聯外網站則自動輸入為cupoy網頁)
            source_data = re.findall('[\S]+com',item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href'))
            if len(source_data)>0:
                data[count]['source'] = source_data[0]
            else:
                data[count]['source'] = 'https://www.cupoy.com'
        
            count +=1
        else:
            continue


    #設定預定要爬取的文章數量並用回存數量作為比對停止器
    if count >= 500:
        switch = False
    else:
        continue
    
    
    #模擬器下拉畫面
    time.sleep(5)
    browser.execute_script("window.scrollBy(0,10000)")    

#關閉虛擬瀏覽器
browser.quit()
df = pd.DataFrame(data)
df = df.T
df.head(500)

,category,link,title,summary,source
1,科技,https://technews.tw/2020/04/16/tsmc-outlook-at...,臺積電維持原有資本支出不變，3 奈米製程預計 2021 年試產 | TechNews 科技新報,"晶圓代工龍頭臺積電 16 日發表 2020 年第 1 季財報，營收金額達到新臺幣 3,106...",https://www.cupoy.com
2,科技,https://technews.tw/2020/04/17/infineon-succes...,英飛凌宣佈完成 87 億美元收購賽普拉斯，將一躍成車用半導體龍頭 | TechNews 科技新報,德國科技大廠英飛凌（Infineon Technologies）宣佈，已經完成收購賽普拉斯半...,https://www.cupoy.com
3,科技,https://technews.tw/2020/04/17/huawei-chip-uni...,憂臺積電受美限制，傳華為晶片擴大轉單中芯 | TechNews 科技新報,路透社 16 日引述數名知情人士報導，中國電信設備巨擘華為因應美國可能祭出更多封殺措施，正逐...,https://www.cupoy.com
4,科技,https://technews.tw/2020/04/17/unpaid-leave-su...,全臺無薪假激增破萬，竹科也不例外 | TechNews 科技新報,由於受到武漢肺炎疫情衝擊，無薪假人數正持續增加，據勞動部統計，全臺已破萬人。\r\r\r\r...,https://www.cupoy.com
5,職場白領,http://sc.piee.pw/HDBAH,「巧妙的」的英文怎麼說？ | 經理人,ingenious 可以用來形容一個人富有智慧、謀略，相似詞有 clever（聰明）、cun...,https://www.cupoy.com
...,...,...,...,...,...
496,科技,https://technews.tw/2020/04/16/tsmc-outlook-at...,臺積電維持原有資本支出不變，3 奈米製程預計 2021 年試產 | TechNews 科技新報,"晶圓代工龍頭臺積電 16 日發表 2020 年第 1 季財報，營收金額達到新臺幣 3,106...",https://www.cupoy.com
497,科技,https://technews.tw/2020/04/17/infineon-succes...,英飛凌宣佈完成 87 億美元收購賽普拉斯，將一躍成車用半導體龍頭 | TechNews 科技新報,德國科技大廠英飛凌（Infineon Technologies）宣佈，已經完成收購賽普拉斯半...,https://www.cupoy.com
498,科技,https://technews.tw/2020/04/17/huawei-chip-uni...,憂臺積電受美限制，傳華為晶片擴大轉單中芯 | TechNews 科技新報,路透社 16 日引述數名知情人士報導，中國電信設備巨擘華為因應美國可能祭出更多封殺措施，正逐...,https://www.cupoy.com
499,科技,https://technews.tw/2020/04/17/unpaid-leave-su...,全臺無薪假激增破萬，竹科也不例外 | TechNews 科技新報,由於受到武漢肺炎疫情衝擊，無薪假人數正持續增加，據勞動部統計，全臺已破萬人。\r\r\r\r...,https://www.cupoy.com


In [54]:
#測試


soup = BeautifulSoup(browser.page_source,'html.parser')
res = soup.find_all('div',{'class':'sc-eEieub sc-iuDHTM ibJqYc'})
#分類或來源
a = res[0].find('div',{'class':'sc-gacfCG bPSpUf'}).string
#文章來源網址
b = res[0].find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href')
#標題
c = res[0].find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('title')
#簡介
d = res[0].find('p',{'class':'sc-FQuPU sc-ciodno bvnzOw'}).text
#來源網址
e = re.findall('[\S]+com',b)

e

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=63796): Max retries exceeded with url: /session/36eb241bd4d189b133325d46307a5228/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022E984FEF08>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。'))